In [1]:
import json
import os
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys
import optparse
import shutil
from mri_preproc.paths import hemond_data

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
os.getcwd()

'/home/srs-9/Projects/ms_mri/monai_analysis/second-train'

In [3]:
dataroot = "/mnt/h/3Tpioneer_bids"
dataset = hemond_data.get_pituitary_3Tpioneer_bids(dataroot)

In [5]:
# infile = dataset.find_scan(subid="ms1001", ses="20180323").image
infile = dataset[0].image
infile

PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1001/ses-20170215/t1.nii.gz')

In [12]:
infile

PosixPath('/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20180323/proc/flair-brain-mni_reg.nii.gz')

In [6]:
datalist = {
    "testing": [{"image": infile.name}]
}

datalist_file = "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f)


task = {
  "name": "infer_pit",
  "task": "segmentation",
  "modality": "MRI",
  "datalist": "datalist.json",
  "dataroot": infile.parent,
}

task_file = os.path.join("/work", "task.json")
with open(task_file, "w") as f:
    json.dump(task, f)


FileNotFoundError: [Errno 2] No such file or directory: '/work/task.json'

In [ ]:
work_dir = "/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir22"  # the algorithm working directory generated by AlgoGen/BundleGen
input_cfg = "/work/task.json"  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)


In [ ]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
pred = ensemble(pred_param = {"image_save_func": {"_target_": "SaveImage", "output_dir": "/work"}})